<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

***

# Юнит 1. Работа с базой данных из Python

1. Импортируем все необходимые нам библиотеки

In [72]:

import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup

2. Создаём соединение с параметрами(укажите ваши параметры)

In [74]:
# connection = psycopg2.connect(
#    dbname=DBNAME,
#    user=USER,
#    host=HOST,
#    password=PASSWORD,
#    port=PORT
# )

3. Запрос

In [75]:
# код запроса представляет из себя строковую переменную
# n = 10
# query = f'''select * 
#             from sql.pokemon 
#             limit {n}
#          '''
# лучше использовать f-строки - так удобнее передавать параметры

4. Выполнение запроса

In [76]:
# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
# df = pd.read_sql_query(query, connection)
# df

5. Финал

In [77]:
# не забываем закрыть соединение после окончания работы
#connection.close()

***

# Юнит 2. Знакомство с данными

1. Наш проект включает в себя несколько этапов:
- знакомство с данными;
- предварительный анализ данных;
- детальный анализ вакансий;
- анализ работодателей;
- предметный анализ.

2. Знакомство с данными
    VACANCIES

    Таблица хранит в себе данные по вакансиям и содержит следующие столбцы:

    ![VACANCIES](data/VACANCIES.png "Vacancies")

    Зарплатная вилка — это верхняя и нижняя граница оплаты труда в рублях (зарплаты в других валютах уже переведены в рубли). Соискателям она показывает, в каком диапазоне компания готова платить сотруднику на этой должности.

    AREAS

    Таблица-справочник, которая хранит код региона и его название.

    ![AREAS](data/AREAS.png "Areas")

    EMPLOYERS

    Таблица-справочник со списком работодателей.

    ![EMPLOYERS](data/EMPLOYERS.png "Employers")

    INDUSTRIES

    Таблица-справочник вариантов сфер деятельности работодателей.

    ![INDUSTRIES](data/INDUSTRIES.png "Industries")

    EMPLOYERS_INDUSTRIES

    Дополнительная таблица, которая существует для организации связи между работодателями и сферами их деятельности.

    Эта таблица нужна нам, поскольку у одного работодателя может быть несколько сфер деятельности (или работодатели могут вовсе не указать их). Для удобства анализа необходимо хранить запись по каждой сфере каждого работодателя в отдельной строке таблицы.

    ![EMPLOYERS_INDUSTRIES](data/EMPLOYERS_INDUSTRIES.png "Employers_Industries")

***

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [78]:
# текст запроса
query_3_1 = f'''select count(*)
                from vacancies
'''

In [79]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\1114379196.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [80]:
# текст запроса
query_3_2 = f'''select count(*)
                from employers
'''

In [81]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
df

C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\3062646138.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_2, connection)


,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [82]:
# текст запроса
query_3_3 = f'''select count(*)
                from areas
'''

In [83]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
df

C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\928149064.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_3, connection)


,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [84]:
# текст запроса
query_3_4 = f'''select count(*)
                from industries
'''

In [85]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
df

C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\2101691108.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_4, connection)


,count
0,294


Выводы по предварительному анализу данных:
- Наша база данных имеет 49 197 вакансий и 294 сферы деятельности

***

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [86]:
# текст запроса
query_4_1 = f'''select a.name area, count(v.id) cnt
                from areas a
                join vacancies v on a.id = v.area_id
                group by 1
                order by 2 desc
                limit 5
'''

In [87]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
df

C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\2457173557.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_1, connection)


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [88]:
# текст запроса
query_4_2 = f'''select count(*)
                from vacancies
                where salary_from is not null or salary_to is not null
'''

In [89]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
df

C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\970655580.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_2, connection)


,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [90]:
# текст запроса
query_4_3_1 = f'''select round(avg(salary_from))
                from vacancies
'''

query_4_3_2 = f'''select round(avg(salary_to))
                from vacancies
'''

In [91]:
# результат запроса
df1 = pd.read_sql_query(query_4_3_1, connection)
display(df1)

df2 = pd.read_sql_query(query_4_3_2, connection)
display(df2)


C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\2350959391.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql_query(query_4_3_1, connection)


,round
0,71065.0


C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\2350959391.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql_query(query_4_3_2, connection)


,round
0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [92]:
# текст запроса
query_4_4 = f'''select schedule, employment, count(*) cnt
                from vacancies
                group by 1, 2
                order by 3 desc
                limit 5
'''

In [93]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
display(df)


C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\3690157244.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_4, connection)


,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [94]:
# текст запроса
query_4_5 = f'''select experience, count(*)
                from vacancies
                group by 1
                order by 2
                limit 5
'''

In [95]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
display(df)

C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\969833449.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_5, connection)


,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Выводы по детальному анализу вакансий:
- Первое место по количеству вакансий занимает Москва(5 333), с отрывом от второго места(Санкт-Петербург) практически в два раза(2851)
- Границы средней зарплаты по вакансиям составляют 71 000 - 110 000 рублей
- Наибольшей популярностью обладает связка "Полный день - Полная занятость" - более 35 000 вакансий. Наименьшей - "Сменный график - Полная занятость" - менее тысячи вакансий

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [96]:
# текст запроса
query_5_1 = f'''select e.name, count(*)
                from employers e
                join vacancies v on e.id = v.employer_id
                group by 1
                order by 2 desc
                limit 5
'''

In [97]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
df

C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\3834399874.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_1, connection)


,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [98]:
# текст запроса
query_5_2 = f'''select a.name, count(e.id) empl_cnt
                from areas a
                left join employers e on a.id = e.area
                left join vacancies v on a.id = v.area_id
                where v.id is null
                group by 1
                order by 2 desc
                limit 10
'''

In [99]:
# результат запроса
df = pd.read_sql_query(query_5_2, connection)
df

C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\2250345976.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_2, connection)


,name,empl_cnt
0,Россия,410
1,Казахстан,207
2,Московская область,75
3,Краснодарский край,19
4,Беларусь,18
5,Ростовская область,18
6,Азербайджан,17
7,Нижегородская область,16
8,Республика Татарстан,16
9,Узбекистан,15


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [100]:
# текст запроса
query_5_3 = f'''select e.id, count( distinct area_id)
                from employers e
                join vacancies v on e.id = v.employer_id 
                group by 1
                order by 2 desc
                limit 5
'''

In [101]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
df

C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\201667002.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_3, connection)


,id,count
0,1740,181
1,2748,152
2,5724811,116
3,5130287,88
4,3682876,71


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [102]:
# текст запроса
query_5_4 = f'''select count(e.id) - count(i.employer_id) cnt
                from employers e 
                left join employers_industries i on e.id = i.employer_id
'''

In [103]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
df

C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\3493105328.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_4, connection)


,cnt
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [104]:
# текст запроса
query_5_5 = f'''select name, count(industry_id) cnt_ind
                from employers_industries i
                join employers e on i.employer_id = e.id
                group by 1
                having count(industry_id) = '4'
                order by 1
                limit 1 offset 2
'''

In [105]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
df

C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\388711819.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_5, connection)


,name,cnt_ind
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [106]:
# текст запроса
query_5_6 = f'''select count(*)
                from industries i
                left join employers_industries  e on e.industry_id = i.id
                where i.name = 'Разработка программного обеспечения'
'''

In [107]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
df

C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\821628714.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_6, connection)


,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [108]:
# код для получения списка городов-милионников
url='https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
cities = []
for i in range(16):
    city = soup.select(f'#mw-content-text > div.mw-content-ltr.mw-parser-output > table.standard.sortable > tbody > tr:nth-child({i+2}) > td:nth-child(2) > a')
    cities.append(city[0].text)
cities = tuple(cities)
print(cities)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')


In [109]:
# текст запроса
query_5_7 = f'''with temp as (select a.name, count(*) cnt
                from vacancies v
                left join areas a on v.area_id = a.id
                left join employers e on e.id = v.employer_id
                where e.name = 'Яндекс' and  a.name in {cities}
                group by 1)
                
                select *
                from temp
                union all
                select 'Total', sum(cnt)
                from temp
                order by 2
'''

In [110]:
# результат запроса
df = pd.read_sql_query(query_5_7, connection)
df

C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\1965596796.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_7, connection)


,name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Самара,26.0
8,Уфа,26.0
9,Краснодар,30.0


Выводы по анализу работодателей:
- Мастодонтом по количеству вакансий является компания "Яндекс", опережая ближайшего работодателя почти в 4 раза
- Также мы можем заметить регионы, в которых нет вакансий, но есть работодатели. Этими регионами являются страны и области

***

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [111]:
# текст запроса
query_6_1 = f'''select count(*)
                from vacancies
                where name ilike '%data%' or name ilike '%данн%'
'''

In [112]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
df

C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\366702258.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_1, connection)


,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [113]:
# текст запроса
query_6_2 = f'''select count(*)
                from vacancies
                where (name ilike '%data scientist%' 
                or name ilike '%data science%'
                or name ilike '%иследователь данных%'
                or name like '%ML%'
                or name ilike '%machine learning%'
                or name ilike '%машинн%обучен%')
                and name not ilike '%HTML%'
                and (name ilike '%junior%'
                or experience = 'Нет опыта'
                or employment = 'Стажировка')
'''

In [114]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
df

C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\2267038862.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_2, connection)


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [115]:
# текст запроса
query_6_3 = f'''select count(*)
                from vacancies
                where (name ilike '%data scientist%' 
                or name ilike '%data science%'
                or name ilike '%иследователь данных%'
                or name like '%ML%'
                or name ilike '%machine learning%'
                or name ilike '%машинн%обучен%')
                and name not ilike '%HTML%'
                and (key_skills like '%SQL%'
                or key_skills like '%postgres%')
'''

In [116]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
df

C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\3794299702.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_3, connection)


,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [117]:
# текст запроса
query_6_4 = f'''select count(*)
                from vacancies
                where (name ilike '%data scientist%' 
                or name ilike '%data science%'
                or name ilike '%иследователь данных%'
                or name like '%ML%'
                or name ilike '%machine learning%'
                or name ilike '%машинн%обучен%')
                and name not ilike '%HTML%'
                and key_skills like '%Python%'
'''

In [118]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
df

C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\4264665179.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_4, connection)


,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [119]:
# текст запроса
query_6_5 = f'''select round(avg(length(key_skills) - length(replace(key_skills, CHR(9), '')) + 1), 2)
                from vacancies
                where (name ilike '%data scientist%' 
                or name ilike '%data science%'
                or name ilike '%иследователь данных%'
                or name like '%ML%'
                or name ilike '%machine learning%'
                or name ilike '%машинн%обучен%')
                and name not ilike '%HTML%'
                and key_skills is not null
'''

In [120]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
df

C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\3654210517.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_5, connection)


,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [121]:
# текст запроса
query_6_6 = f'''with temp as (select *, (coalesce(salary_from, 0) + coalesce(salary_to, 0))/2 mean
                            from vacancies
                            where (name ilike '%data scientist%' 
                            or name ilike '%data science%'
                            or name ilike '%иследователь данных%'
                            or name like '%ML%'
                            or name ilike '%machine learning%'
                            or name ilike '%машинн%обучен%')
                            and name not ilike '%HTML%'
                            and (salary_from is not null
                            and salary_to is not null)
                        
                            union all
                        
                            select *, (coalesce(salary_from, 0) + coalesce(salary_to, 0)) mean
                            from vacancies
                            where (name ilike '%data scientist%' 
                            or name ilike '%data science%'
                            or name ilike '%иследователь данных%'
                            or name like '%ML%'
                            or name ilike '%machine learning%'
                            or name ilike '%машинн%обучен%')
                            and name not ilike '%HTML%'
                            and ((salary_from is not null
                            and salary_to is null) 
                            or (salary_from is null
                            and salary_to is not null)))
                select experience, round(avg(mean)) avg_salary
                from temp
                where experience = 'От 3 до 6 лет'  
                group by 1
                
'''

In [122]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
df

C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\519194770.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_6, connection)


,experience,avg_salary
0,От 3 до 6 лет,243115.0


Выводы по предметному анализу:
- К изучению и работе с данными имеет отношение 1771 вакансия, но из них специалистам Junior по Data Science подходит только 51 вариант. Это обусловлено тем, что компаниям выгоднее брать более опытных специалистов, которые имеют в среднем 6 ключевых навыков для DS. Но, наработав опыт(от 3 до 6 лет) у нас появится возможность рассчитывать на высокую среднюю зарплату(243 000 руб.)

Давайте поможем найти нашей знакомой подходящую работу. Условия такие: удаленный тип рабочего графика, частичная занятость, отсутствие опыта работы, а так же, средняя заработная плата, если указаны обе вилки зп, или же просто та вилка, которая указана единственной. В выводе должны быть следующие колонки: название вакансии, ключевые навыки, работодатель, город и средняя ЗП

In [123]:
# текст запроса
query = f'''select v.name vacanc, v.key_skills, e.name employ, a.name area, (coalesce(salary_from, 0) + coalesce(salary_to, 0))/2 avg_salary
            from vacancies v
            left join employers e on e.id = v.employer_id
            left join areas a on a.id = v.area_id
            where v.schedule = 'Удаленная работа'
            and v.employment = 'Частичная занятость'
            and v.experience = 'Нет опыта'
            and (salary_from is not null
            and salary_to is not null)
            
            
            union
            
            select v.name vacanc, v.key_skills, e.name employ, a.name area, (coalesce(salary_from, 0) + coalesce(salary_to, 0)) avg_salary
            from vacancies v
            left join employers e on e.id = v.employer_id
            left join areas a on a.id = v.area_id
            where v.salary_to is not null
            and v.schedule = 'Удаленная работа'
            and v.employment = 'Частичная занятость'
            and v.experience = 'Нет опыта'
            and ((salary_from is not null
            and salary_to is null) 
            or (salary_from is null
            and salary_to is not null))
            
            order by 5 desc
            limit 15
            
            
'''

In [124]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\fdgod\AppData\Local\Temp\ipykernel_17348\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,vacanc,key_skills,employ,area,avg_salary
0,Автор по информационной безопасности,Английский язык\tПользователь ПК\t1С: Документ...,Ворк5,Омск,100000
1,Автор по информационной безопасности,Английский язык\tПользователь ПК\t1С: Документ...,Ворк5,Ростов-на-Дону,100000
2,Автор по информационной безопасности,Английский язык\tПользователь ПК\t1С: Документ...,Ворк5,Нур-Султан,100000
3,Автор по информационной безопасности,Английский язык\tПользователь ПК\t1С: Документ...,Ворк5,Челябинск,100000
4,Автор по программированию,Английский язык\tПользователь ПК\t1С: Документ...,Ворк5,Нур-Султан,100000
5,Автор по программированию,Английский язык\tПользователь ПК\t1С: Документ...,Ворк5,Самара,100000
6,Автор по программированию,Английский язык\tПользователь ПК\t1С: Документ...,Ворк5,Ростов-на-Дону,100000
7,Автор по информационной безопасности,Английский язык\tПользователь ПК\t1С: Документ...,Ворк5,Самара,100000
8,Автор по программированию,Английский язык\tПользователь ПК\t1С: Документ...,Ворк5,Челябинск,100000
9,Автор по программированию,Английский язык\tПользователь ПК\t1С: Документ...,Ворк5,Омск,100000


***

# Общий вывод по проекту

- Мы заверишили исследование нашего второго проекта, основанного на вакансиях работодателей на сайте hh.ru
- Мы успешно объединяли разные таблицы по ключевому признаку, делали выборки, группировки данных с помощью SQL и СУБД PostgreSQL

In [125]:
#закрываем соединение с СУБД
connection.close()